In [24]:
import numpy as np
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily, Hourly
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error, mean_absolute_error

location = Point(-6.9389, 107.7528)
start = datetime(2023, 1, 1, 0, 0, 0)
short_term_end = datetime(2023, 1, 7, 23, 59, 59) # 7 days
medium_term_end = datetime(2023, 3, 31, 23, 59, 59) # 3 months
long_term_end = datetime(2023, 12, 31, 23, 59, 59) # 1 year

daily_long_data = Daily(location, start, long_term_end)
daily_long_data = daily_long_data.fetch()

hourly_long_data = Hourly(location, start, long_term_end)
hourly_long_data = hourly_long_data.fetch()

daily_long_temp = daily_long_data['tavg']
daily_medium_temp = daily_long_temp[start:medium_term_end]
hourly_long_temp = hourly_long_data['temp']
hourly_medium_temp = hourly_long_temp[start:medium_term_end]
hourly_short_temp = hourly_long_temp[start:short_term_end]
print(daily_long_temp)
print(hourly_long_temp)
print("\nMissing data:")
print("Daily:", daily_long_temp.isnull().sum())
print("Hourly:", hourly_long_temp.isnull().sum())

time
2023-01-01    21.4
2023-01-02    22.4
2023-01-03    21.8
2023-01-04    21.8
2023-01-05    22.2
              ... 
2023-12-27    22.6
2023-12-28    23.5
2023-12-29    23.4
2023-12-30    22.3
2023-12-31    22.3
Freq: D, Name: tavg, Length: 365, dtype: float64
time
2023-01-01 00:00:00    19.9
2023-01-01 01:00:00    20.9
2023-01-01 02:00:00    21.7
2023-01-01 03:00:00    23.6
2023-01-01 04:00:00    23.2
                       ... 
2023-12-31 19:00:00    21.3
2023-12-31 20:00:00    19.6
2023-12-31 21:00:00    20.8
2023-12-31 22:00:00    20.4
2023-12-31 23:00:00    19.2
Freq: h, Name: temp, Length: 8760, dtype: float64

Missing data:
Daily: 0
Hourly: 0


In [29]:
def introduce_missingness(data, rate, pattern="scattered", block_size=12):
    data_with_nan = data.copy()
    n = len(data)
    missing_indices = []
    if pattern == "scattered":
        missing_indices = np.random.choice(n, int(rate * n), replace=False)
    elif pattern == "block":
        num_blocks = int(rate * n / block_size)
        for _ in range(num_blocks):
            start_idx = np.random.randint(0, n - block_size)
            missing_indices.extend(range(start_idx, start_idx + block_size))
    data_with_nan.iloc[missing_indices] = np.nan
    return data_with_nan

missing_rates = [0.1, 0.2, 0.3]
patterns = ["scattered", "block"]

def create_missing_datasets(data, patterns, rates, block_size):
    datasets = {}
    for pattern in patterns:
        for rate in rates:
            key = f"{pattern}_{int(rate*100)}"
            datasets[key] = introduce_missingness(data, rate, pattern, block_size)
    return datasets

hourly_missing_data_long = create_missing_datasets(hourly_long_temp, patterns, missing_rates, block_size=12)
daily_missing_data_long = create_missing_datasets(daily_long_temp, patterns, missing_rates, block_size=7)
hourly_missing_data_medium = create_missing_datasets(hourly_medium_temp, patterns, missing_rates, block_size=12)
daily_missing_data_medium = create_missing_datasets(daily_medium_temp, patterns, missing_rates, block_size=7)
hourly_missing_data_short = create_missing_datasets(hourly_short_temp, patterns, missing_rates, block_size=12)


In [45]:
print("\nMissing data:")

print("Long-Hourly:")
print(f"  original: {hourly_long_temp.notnull().sum()}")
for key, dataset in hourly_missing_data_long.items():
    print(f"  {key}: {dataset.isnull().sum().sum()} missing values")

print("Long-Daily:")
print(f"  original: {daily_long_temp.notnull().sum()}")
for key, dataset in daily_missing_data_long.items():
    print(f"  {key}: {dataset.isnull().sum().sum()} missing values")

print("Medium-Hourly:")
print(f"  original: {hourly_medium_temp.notnull().sum()}")
for key, dataset in hourly_missing_data_medium.items():
    print(f"  {key}: {dataset.isnull().sum().sum()} missing values")

print("Medium-Daily:")
print(f"  original: {daily_medium_temp.notnull().sum()}")
for key, dataset in daily_missing_data_medium.items():
    print(f"  {key}: {dataset.isnull().sum().sum()} missing values")

print("Short-Hourly:")
print(f"  original: {hourly_short_temp.notnull().sum()}")
for key, dataset in hourly_missing_data_short.items():
    print(f"  {key}: {dataset.isnull().sum().sum()} missing values")



Missing data:
Long-Hourly:
  original: 8760
  scattered_10: 876 missing values
  scattered_20: 1752 missing values
  scattered_30: 2628 missing values
  block_10: 845 missing values
  block_20: 1599 missing values
  block_30: 2307 missing values
Long-Daily:
  original: 365
  scattered_10: 36 missing values
  scattered_20: 73 missing values
  scattered_30: 109 missing values
  block_10: 35 missing values
  block_20: 61 missing values
  block_30: 86 missing values
Medium-Hourly:
  original: 2160
  scattered_10: 216 missing values
  scattered_20: 432 missing values
  scattered_30: 648 missing values
  block_10: 203 missing values
  block_20: 381 missing values
  block_30: 562 missing values
Medium-Daily:
  original: 90
  scattered_10: 9 missing values
  scattered_20: 18 missing values
  scattered_30: 27 missing values
  block_10: 7 missing values
  block_20: 14 missing values
  block_30: 15 missing values
Short-Hourly:
  original: 168
  scattered_10: 16 missing values
  scattered_20: 33 

In [34]:
def cubic_spline_interpolation(original_data, missing_data):
    interpolated_data = missing_data.copy()
    observed_indices = ~missing_data.isna()
    observed_x = np.arange(len(original_data))[observed_indices]
    observed_y = original_data[observed_indices]

    spline = CubicSpline(observed_x, observed_y)
    interpolated_data[~observed_indices] = spline(np.arange(len(original_data))[~observed_indices])

    return interpolated_data

In [ ]:
def evaluate_interpolation(original_data, interpolated_data, missing_data):
    true_values = original_data[missing_data.isna()]
    imputed_values = interpolated_data[missing_data.isna()]

    rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
    mae = mean_absolute_error(true_values, imputed_values)
    return rmse, mae
results = []
for dataset_type, datasets, original_data in zip(
    ["Hourly", "Daily", "Hourly", "Daily", "Hourly"],
    [
        hourly_missing_data_long, daily_missing_data_long,
        hourly_missing_data_medium, daily_missing_data_medium,
        hourly_missing_data_short
    ],
    [
        hourly_long_temp, daily_long_temp,
        hourly_medium_temp, daily_medium_temp,
        hourly_short_temp
    ]
):
    granularity = (
        "long" if id(original_data) in [id(hourly_long_temp), id(daily_long_temp)] else
        "medium" if id(original_data) in [id(hourly_medium_temp), id(daily_medium_temp)] else
        "short"
    )
    for key, missing_data in datasets.items():
        interpolated_data = cubic_spline_interpolation(original_data, missing_data)
        rmse, mae = evaluate_interpolation(original_data, interpolated_data, missing_data)
        results.append([dataset_type, key.split("_")[0], key.split("_")[1], granularity, rmse, mae])

results_df = pd.DataFrame(results, columns=["Type", "Pattern", "Rate (%)", "Granularity", "RMSE", "MAE"])
print(results_df)

      Type    Pattern Rate (%) Granularity      RMSE       MAE
0   Hourly  scattered       10        long  1.927739  1.446166
1   Hourly  scattered       20        long  2.158186  1.623185
2   Hourly  scattered       30        long  2.606162  1.803609
3   Hourly      block       10        long  6.730402  5.252770
4   Hourly      block       20        long  8.126094  5.946614
5   Hourly      block       30        long  9.055209  6.346494
6    Daily  scattered       10        long  0.479914  0.397070
7    Daily  scattered       20        long  0.529246  0.381967
8    Daily  scattered       30        long  0.771659  0.566188
9    Daily      block       10        long  0.679802  0.563758
10   Daily      block       20        long  1.900691  1.543932
11   Daily      block       30        long  0.965083  0.850583
12  Hourly  scattered       10      medium  2.119266  1.656264
13  Hourly  scattered       20      medium  2.255280  1.723923
14  Hourly  scattered       30      medium  2.479978  1